In [1]:
import numpy as np
import datetime
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import yfinance as yf


In [10]:
def download_data(stock,start,end):
    data={}
    ticker=yf.download(stock,start,end)
    data['Close'] = ticker['Adj Close']
    return pd.DataFrame(data)


In [13]:
def construct_features(data,lags=2):
    for i in range(0,lags):
        data['Lag%s' % str(i+1)] = data['Close'].shift(i+1)
    
    data['Today Change'] = data['Close'].pct_change() * 100
    
    #Normalising the data for the logistic regression and creating 
    # a target vector of -1 / 1 
    
    for i in range(0,lags):
        data['Lag%s' %str(i+1)] = data['Lag%s' % str(i+1)].pct_change()
    data['Direction'] = np.where(data['Today Change'] > 0, 1, -1)
        
    print(data)

In [16]:
start_date = datetime.datetime(2017,1,1)
end_date = datetime.datetime(2018,1,1)

stock_data = download_data('IBM',start_date,end_date)

construct_features(stock_data)

stock_data.dropna(inplace=True)
stock_data


[*********************100%***********************]  1 of 1 completed
                 Close      Lag1      Lag2  Today Change  Direction
Date                                                               
2017-01-03  134.699112       NaN       NaN           NaN         -1
2017-01-04  136.366821       NaN       NaN      1.238100          1
2017-01-05  135.915649  0.012381       NaN     -0.330852         -1
2017-01-06  136.584320 -0.003309  0.012381      0.491975          1
2017-01-09  135.069687  0.004920 -0.003309     -1.108936         -1
...                ...       ...       ...           ...        ...
2017-12-22  127.629326 -0.009480 -0.001827      0.660091          1
2017-12-26  127.905495  0.006601 -0.009480      0.216384          1
2017-12-27  128.156570  0.002164  0.006601      0.196298          1
2017-12-28  128.918152  0.001963  0.002164      0.594259          1
2017-12-29  128.399261  0.005943  0.001963     -0.402496         -1

[251 rows x 5 columns]


,Close,Lag1,Lag2,Today Change,Direction
Date,,,,,
2017-01-06,136.584320,-0.003309,0.012381,0.491975,1
2017-01-09,135.069687,0.004920,-0.003309,-1.108936,-1
2017-01-10,133.353622,-0.011089,0.004920,-1.270503,-1
2017-01-11,135.150253,-0.012705,-0.011089,1.347268,1
2017-01-12,135.311401,0.013473,-0.012705,0.119236,1
...,...,...,...,...,...
2017-12-22,127.629326,-0.009480,-0.001827,0.660091,1
2017-12-26,127.905495,0.006601,-0.009480,0.216384,1
2017-12-27,128.156570,0.002164,0.006601,0.196298,1


In [19]:
X = stock_data[['Lag1','Lag2']]
y = stock_data['Direction']

X_train, X_test ,y_train, y_test = train_test_split(X,y,test_size=0.3)

In [24]:
model = LogisticRegression()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

print('Accuracy of the model: %.2f' % accuracy_score(y_test,predictions))
print(confusion_matrix(predictions,y_test))
# pretty terrible

Accuracy of the model: 0.51
[[ 0  0]
 [37 38]]
